In [61]:
%run Nonlinear_Dynamics
import numpy as np
import pylab as pl
def solve_sdof(max_time=10.0, g = 9.81,l = 1,m = 1,zeta = 0.1, A = 3.78, w = 2, x0 = 0, v0 = 0, plotnow = 1):

    
    def sdof_deriv(x1_x2, t, g = 9.81,l = 1,m = 1,zeta = 0.1,A = 3.78, w = 2):
        """Compute the time-derivative of a SDOF system."""
        x1, x2 = x1_x2
        return [x2, -zeta/m/l*x2 - g/l*np.sin(x1) + A*np.cos(w*t)]

    x0i=((x0, v0))
    # Solve for the trajectories
    t = sp.linspace(0, max_time, int(250*max_time))
    x_t = sp.integrate.odeint(sdof_deriv, x0i, t)
    
    x, v = x_t.T
    f = A*np.cos(w*t)
    
    if plotnow == 1:
        #fig = plt.figure()
        #ax = fig.add_axes([0, 0, 1, 1], projection='3d')
        plt.plot(t,x,t,f,'--')
        pl.legend(['x','Forcing Function'])
        plt.xlabel('Time (s)')
        plt.ylabel('x')
        plt.title('Integrated Response of the Damped, Nonlinear Pendulum')
        plt.show()

    return t, x, v


In [62]:
solve_sdof(max_time=3*10*np.pi, x0 = 0, v0 = 0, plotnow = 1)

(array([  0.00000000e+00,   4.00033020e-03,   8.00066041e-03, ...,
          9.42397789e+01,   9.42437793e+01,   9.42477796e+01]),
 array([  0.00000000e+00,   3.02402364e-05,   1.20938577e-04, ...,
          7.21703459e-01,   7.21888224e-01,   7.22029675e-01]),
 array([ 0.        ,  0.01511765,  0.03022593, ...,  0.05160136,
         0.04077356,  0.02994626]))

In [63]:
#From nonlinear.py posted by Daniel Clark
import pylab as pl
import numpy as np
import scipy.optimize as sci
import scipy.integrate as sp

# this code is the nonlinear case of \dotdot{x} + zeta/m/l*\dot{x} + g/l*sin(x) = A*cos(w*t)

def PendulumTimeSeriesResults(N = 99,g = 9.81,l = 1,m = 1,zeta = 0.1,w = 2,A = 3.75):
    t = np.linspace(0, 10*np.pi, N+1)    # time samples of forcing function
    t = t[0:-1]                          # Removing the extra sample
    f = A*np.cos(w*t)                    # My forcing function
    T = t[-1]
    xbar = 0.7/3.78*f

    def FUNCTION(xbar):
        N = len(xbar)
        Xbar = np.fft.fft(xbar)
        omega = np.fft.fftfreq(N, T/(2*np.pi*N) )# + 0.0000001 # list of frequencies
        dotxbar = np.fft.ifft(np.multiply((1j*omega),Xbar))
        dotdotxbar = np.fft.ifft(np.multiply((1j*omega)**2,Xbar))
        R = dotdotxbar + zeta/m/l*dotxbar + g/l*np.sin(xbar) - f
        R = R**2
        R = np.sum(R)
        return R
    print(g)
    optimizedResults = sci.minimize(FUNCTION, xbar, method='SLSQP')
    xbar = optimizedResults.x

    print(optimizedResults)
    print(xbar)

    pl.plot(t,xbar,t,f,'--')
    pl.legend(['x','Forcing Function'])
    pl.xlabel('Time (s)')
    pl.title('Harmonic Balance Method Response of the Damped, Nonlinear Pendulum')
    pl.show()

In [64]:
PendulumTimeSeriesResults(N=300)
#Although we can obtain a solution for the damped, nonlinear pendulum, it only captures the harmonic response
#i.e. The pendulum cannot flip all the way around or 'jump' to another solution

C:\Users\tidba_000\Anaconda3\lib\site-packages\scipy\optimize\slsqp.py:62: ComplexWarning: Casting complex values to real discards the imaginary part
  jac[i] = (func(*((x0+dx,)+args)) - f0)/epsilon
C:\Users\tidba_000\Anaconda3\lib\site-packages\scipy\optimize\slsqp.py:407: ComplexWarning: Casting complex values to real discards the imaginary part
  slsqp(m, meq, x, xl, xu, fx, c, g, a, acc, majiter, mode, w, jw)


9.81
  status: 9
     jac: array([ -28.18160342,   39.95926279,  -73.90018368,   92.46558085,
        -92.88466898,  101.40849441, -105.13775692,  105.89677048,
        -97.87773889,   76.61110686,  -58.92367157,   50.00325548,
        -57.95103911,   37.95865541,   -5.49081653,   -2.80684056,
         -2.08533868,   12.54093476,   -7.00456937,  -19.5166342 ,
         29.13714521,  -35.40851455,   30.40199572,  -35.01536177,
         44.62892592,  -46.27369864,   34.10314193,  -12.76808107,
         13.86925901,  -36.28642672,   40.72153203,  -31.2544179 ,
          0.93640767,   15.04294696,  -11.59417427,   15.90734737,
        -24.48102085,   37.18599164,  -36.76825205,   21.97007632,
        -16.56744193,   16.12144522,  -29.06439766,   20.38087562,
          9.66084081,  -26.67880819,   18.53720488,    3.24856005,
         -5.79221587,  -12.30279903,   15.05744725,  -14.79383986,
          3.24688228,   -7.1686777 ,   20.71095733,  -33.48874419,
         40.42788971,  -29.2367914 